## 4-3,AutoGraph的使用规范
有三种计算图的构建方式：静态计算图，动态计算图，以及Autograph。

TensorFlow 2.0主要使用的是动态计算图和Autograph。

动态计算图易于调试，编码效率较高，但执行效率偏低。

静态计算图执行效率很高，但较难调试。

而Autograph机制可以将动态图转换成静态计算图，兼收执行效率和编码效率之利。

当然Autograph机制能够转换的代码并不是没有任何约束的，有一些编码规范需要遵循，否则可能会转换失败或者不符合预期。

我们将着重介绍Autograph的编码规范和Autograph转换成静态图的原理。

并介绍使用tf.Module来更好地构建Autograph。

本篇我们介绍使用Autograph的编码规范。

### 一，Autograph编码规范总结
1，被@tf.function修饰的函数应尽可能使用TensorFlow中的函数而不是Python中的其他函数。例如使用tf.print而不是print，使用tf.range而不是range，使用tf.constant(True)而不是True.

2，避免在@tf.function修饰的函数内部定义tf.Variable.

3，被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等数据结构变量。

### 二，Autograph编码规范解析

#### 1，被@tf.function修饰的函数应尽量使用TensorFlow中的函数而不是Python中的其他函数。

In [2]:
import tensorflow as tf
import numpy as np

In [11]:
@tf.function
def np_random():
    a = np.random.randn(3,3)
    tf.print(a)
    
@tf.function
def tf_random():
    a = tf.random.normal((3,3))
    tf.print(a)

In [8]:
np_random()
np_random()  # 每次运行结果一样

array([[ 1.09905625, -0.85723539,  0.24951243],
       [ 0.17632273,  0.28894297, -1.10911201],
       [ 0.66196324,  0.58626835,  2.09769295]])
array([[ 1.09905625, -0.85723539,  0.24951243],
       [ 0.17632273,  0.28894297, -1.10911201],
       [ 0.66196324,  0.58626835,  2.09769295]])


In [12]:
tf_random()
tf_random()   # 每次运行结果不一样

[[1.14563155 -0.286019385 -0.643864036]
 [0.922883689 0.875162065 1.82749355]
 [2.7475822 -1.60024977 0.0453699231]]
[[-0.850969613 -0.284239233 1.08223677]
 [1.41694534 0.0228817351 0.828645229]
 [-0.427852273 0.234575346 -0.852913]]


#### 2，避免在@tf.function修饰的函数内部定义tf.Variable.

In [15]:
# 避免在@tf.function修饰的函数内部定义tf.Variable.

x = tf.Variable(1.0, dtype=tf.float32)

@tf.function
def outer_var():
    x.assign_add(1.0)
    tf.print(x)
    return x

outer_var()
outer_var()

2
3


<tf.Tensor: shape=(), dtype=float32, numpy=3.0>

In [17]:
@tf.function
def inner_var():
    x = tf.Variable(1.0, dtype=tf.float32)
    x.assign_add(1.0)
    tf.print(x)
    return x

inner_var()   # 报错

ValueError: in converted code:

    <ipython-input-16-f19252771cb1>:3 inner_var  *
        x = tf.Variable(1.0, dtype=tf.float32)
    /Users/ting/Library/Python/3.7/lib/python/site-packages/tensorflow_core/python/ops/variables.py:260 __call__
        return cls._variable_v2_call(*args, **kwargs)
    /Users/ting/Library/Python/3.7/lib/python/site-packages/tensorflow_core/python/ops/variables.py:254 _variable_v2_call
        shape=shape)
    /Users/ting/Library/Python/3.7/lib/python/site-packages/tensorflow_core/python/ops/variables.py:65 getter
        return captured_getter(captured_previous, **kwargs)
    /Users/ting/Library/Python/3.7/lib/python/site-packages/tensorflow_core/python/eager/def_function.py:502 invalid_creator_scope
        "tf.function-decorated function tried to create "

    ValueError: tf.function-decorated function tried to create variables on non-first call.


#### 3,被@tf.function修饰的函数不可修改该函数外部的Python列表或字典等结构类型变量。

In [20]:
tensor_list = []

# @tf.function
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)
tf.print(tensor_list)

[<tf.Tensor: shape=(), dtype=float32, numpy=5.0>, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>]
[5, 6]


In [22]:
tensor_list = []
@tf.function  #加上这一行切换成Autograph结果将不符合预期
def append_tensor(x):
    tensor_list.append(x)
    return tensor_list

append_tensor(tf.constant(5.0))
append_tensor(tf.constant(6.0))
print(tensor_list)  # 不能对其进行修改

[<tf.Tensor 'x:0' shape=() dtype=float32>]
